# GPT PROMPT GENERATOR

To generate a prompt:
1. In the SECOND cell, add your OpenAI key.
2. In the FOURTH cell, if you have GPT-4 access, you're ready to move on. If not, change CANDIDATE_MODEL='gpt-4' to CANDIDATE_MODEL='gpt-3.5-turbo'
3. In the FOURTH cell, change temperature, max_tokens and number_of_prompts
4. In the last but one cell, fill in the description of your task, up to 15 test cases, and the number of prompts to generate.
5. Now all you need is to run all the cells. Good luck and have fun.

In [19]:
!pip install openai prettytable tqdm tenacity wandb -qq

In [20]:
from prettytable import PrettyTable
import time
import openai
from tqdm import tqdm
import itertools
import wandb
from tenacity import retry, stop_after_attempt, wait_exponential

openai.api_key = "" # enter your OpenAI API key here

use_wandb = True # Weights & Biases (wandb) is a tool used for machine learning experiment tracking, dataset versioning, and model management. if u dont understand, keep it False.

In [21]:
system_gen_system_prompt = """Your job is to generate system prompts for GPT-4, given a description of the use-case and some test cases.

The prompts you will be generating will be for freeform tasks, such as generating a landing page headline, an intro paragraph, solving a math problem, etc.

In your generated prompt, you should describe how the AI should behave in plain English. Include what it will see, and what it's allowed to output. Be creative with prompts to get the best possible results. The AI knows it's an AI -- you don't need to tell it this.

You will be graded based on the performance of your prompt... but don't cheat! You cannot include specifics about the test cases in your prompt. Any prompts with examples will be disqualified.

Most importantly, output NOTHING but the prompt. Do not include anything else in your message."""


ranking_system_prompt = """Your job is to rank the quality of two outputs generated by different prompts. The prompts are used to generate a response for a given task.

You will be provided with the task description, the test prompt, and two generations - one for each system prompt.

Rank the generations in order of quality. If Generation A is better, respond with 'A'. If Generation B is better, respond with 'B'.

Remember, to be considered 'better', a generation must not just be good, it must be noticeably superior to the other.

Also, keep in mind that you are a very harsh critic. Only rank a generation as better if it truly impresses you more than the other.

Respond with your ranking, and nothing else. Be fair and unbiased in your judgement."""

In [22]:
# K is a constant factor that determines how much ratings change
K = 32

CANDIDATE_MODEL = 'gpt-4' # CHANGE HERE IF U NEED
CANDIDATE_MODEL_TEMPERATURE = 0.9

GENERATION_MODEL = 'gpt-3.5-turbo'
GENERATION_MODEL_TEMPERATURE = 0.8
GENERATION_MODEL_MAX_TOKENS = 60

N_RETRIES = 3  # number of times to retry a call to the ranking model if it fails
RANKING_MODEL = 'gpt-3.5-turbo'
RANKING_MODEL_TEMPERATURE = 0.5

NUMBER_OF_PROMPTS = 7 # this determines how many candidate prompts to generate. The higher, the more expensive, but the better the results will be

WANDB_PROJECT_NAME = "LaunayViolette" # used if use_wandb is True, Weights &| Biases project name
WANDB_RUN_NAME = None # used if use_wandb is True, optionally set the Weights & Biases run name to identify this run

In [23]:
def start_wandb_run():
  # start a new wandb run and log the config
  wandb.init(
    project=WANDB_PROJECT_NAME,
    name=WANDB_RUN_NAME,
    config={
      "K": K,
      "system_gen_system_prompt": system_gen_system_prompt,
      "ranking_system_prompt": ranking_system_prompt,
      "candiate_model": CANDIDATE_MODEL,
      "candidate_model_temperature": CANDIDATE_MODEL_TEMPERATURE,
      "generation_model": GENERATION_MODEL,
      "generation_model_temperature": GENERATION_MODEL_TEMPERATURE,
      "generation_model_max_tokens": GENERATION_MODEL_MAX_TOKENS,
      "n_retries": N_RETRIES,
      "ranking_model": RANKING_MODEL,
      "ranking_model_temperature": RANKING_MODEL_TEMPERATURE,
      "number_of_prompts": NUMBER_OF_PROMPTS
      })

  return

In [24]:
# Optional logging to Weights & Biases to reocrd the configs, prompts and results
if use_wandb:
  start_wandb_run()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [25]:
def generate_candidate_prompts(description, test_cases, number_of_prompts):
  outputs = openai.ChatCompletion.create(
      model=CANDIDATE_MODEL, # change this to gpt-3.5-turbo if you don't have GPT-4 access
      messages=[
          {"role": "system", "content": system_gen_system_prompt},
          {"role": "user", "content": f"Here are some test cases:`{test_cases}`\n\nHere is the description of the use-case: `{description.strip()}`\n\nRespond with your prompt, and nothing else. Be creative."}
          ],
      temperature=CANDIDATE_MODEL_TEMPERATURE,
      n=number_of_prompts)

  prompts = []

  for i in outputs.choices:
    prompts.append(i.message.content)
  return prompts

def expected_score(r1, r2):
    return 1 / (1 + 10**((r2 - r1) / 400))

def update_elo(r1, r2, score1):
    e1 = expected_score(r1, r2)
    e2 = expected_score(r2, r1)
    return r1 + K * (score1 - e1), r2 + K * ((1 - score1) - e2)

# Get Score - retry up to N_RETRIES times, waiting exponentially between retries.
@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_score(description, test_case, pos1, pos2, ranking_model_name, ranking_model_temperature):
    score = openai.ChatCompletion.create(
        model=ranking_model_name,
        messages=[
            {"role": "system", "content": ranking_system_prompt},
            {"role": "user", "content": f"""Task: {description.strip()}
Prompt: {test_case['prompt']}
Generation A: {pos1}
Generation B: {pos2}"""}
        ],
        logit_bias={
              '32': 100,  # 'A' token
              '33': 100,  # 'B' token
        },
        max_tokens=1,
        temperature=ranking_model_temperature,
    ).choices[0].message.content
    return score

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_generation(prompt, test_case):
    generation = openai.ChatCompletion.create(
        model=GENERATION_MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"{test_case['prompt']}"}
        ],
        max_tokens=GENERATION_MODEL_MAX_TOKENS,
        temperature=GENERATION_MODEL_TEMPERATURE,
    ).choices[0].message.content
    return generation

def test_candidate_prompts(test_cases, description, prompts):
  # Initialize each prompt with an ELO rating of 1200
  prompt_ratings = {prompt: 1200 for prompt in prompts}

  # Calculate total rounds for progress bar
  total_rounds = len(test_cases) * len(prompts) * (len(prompts) - 1) // 2

  # Initialize progress bar
  pbar = tqdm(total=total_rounds, ncols=70)

  # For each pair of prompts
  for prompt1, prompt2 in itertools.combinations(prompts, 2):
      # For each test case
      for test_case in test_cases:
          # Update progress bar
          pbar.update()

          # Generate outputs for each prompt
          generation1 = get_generation(prompt1, test_case)
          generation2 = get_generation(prompt2, test_case)

          # Rank the outputs
          score1 = get_score(description, test_case, generation1, generation2, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)
          score2 = get_score(description, test_case, generation2, generation1, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)

          # Convert scores to numeric values
          score1 = 1 if score1 == 'A' else 0 if score1 == 'B' else 0.5
          score2 = 1 if score2 == 'B' else 0 if score2 == 'A' else 0.5

          # Average the scores
          score = (score1 + score2) / 2

          # Update ELO ratings
          r1, r2 = prompt_ratings[prompt1], prompt_ratings[prompt2]
          r1, r2 = update_elo(r1, r2, score)
          prompt_ratings[prompt1], prompt_ratings[prompt2] = r1, r2

          # Print the winner of this round
          if score > 0.5:
              print(f"Winner: {prompt1}")
          elif score < 0.5:
              print(f"Winner: {prompt2}")
          else:
              print("Draw")

  # Close progress bar
  pbar.close()

  return prompt_ratings


def generate_optimal_prompt(description, test_cases, number_of_prompts, use_wandb=False):
  if use_wandb:
    wandb_table = wandb.Table(columns=["Prompt", "Ranking"])
    if wandb.run is None:
      start_wandb_run()

  prompts = generate_candidate_prompts(description, test_cases, number_of_prompts)
  prompt_ratings = test_candidate_prompts(test_cases, description, prompts)

  # Print the final ELO ratingsz
  table = PrettyTable()
  table.field_names = ["Prompt", "Rating"]
  for prompt, rating in sorted(prompt_ratings.items(), key=lambda item: item[1], reverse=True):
      table.add_row([prompt, rating])
      if use_wandb:
         wandb_table.add_data(prompt, rating)

  if use_wandb: # log the results to a Weights & Biases table and finsih the run
    wandb.log({"prompt_ratings": wandb_table})
    wandb.finish()
  print(table)

# In the cell below, fill in your description and test cases

In [26]:
description = "Given a prompt, generate a prompt to help large language model understand what user says and have better empathy and generate better responses"

test_cases = [
    {
        'prompt': 'You are a very skilled psychology assistant with extensive knowledge of psychology and intimate relationship problem solving skills, and now you need to talk to users. Please empathise with your users and solve their problems as best as you can.'
    }
]

# test_cases format
# test_cases = [
#     {
#         'prompt': '总会有地上的生灵，敢于直面雷霆的威光',
#     },
#     {
#         'prompt': '若你困于无风之地，我将为你奏响高天之歌',
#     },
#     {
#         'prompt': '倘若七神心中各有标尺，将神之眼赐予众生',
#     }
# ]

if use_wandb:
    wandb.config.update({"description": description,
                        "test_cases": test_cases})

In [27]:
generate_optimal_prompt(description, test_cases, NUMBER_OF_PROMPTS, use_wandb)

 10%|███▏                              | 2/21 [00:04<00:40,  2.12s/it]

Winner: When you receive a prompt, imagine you're a highly trained psychology assistant with deep knowledge in the field and a proven ability to navigate complex emotional and relationship-based issues. Your responsibility is to engage with users, empathize with their situations, interpret their messages with compassion and astuteness, and provide meaningful advice or solutions to their problems. Approach each interaction as if you are guiding someone through a difficult time, and remember, your responses need to reflect a deep understanding and empathy towards their situation.


 14%|████▊                             | 3/21 [00:08<00:52,  2.94s/it]

Winner: Given that you are an advanced AI with a deep understanding of psychology, you must communicate empathetically and effectively with the users. Your aim is to understand the nuances of their problems and provide the best possible solution or advice. Remember to demonstrate a high level of emotional intelligence, showing empathy, respect, and understanding in all interactions. Display active listening skills, ask relevant questions for clarity, and validate the user's feelings to build trust. Use your extensive knowledge of psychology to analyze the issues they present, identify potential causes and recommend feasible solutions. Please note that your advice should not replace professional help but should guide users towards getting the help they need.


 19%|██████▍                           | 4/21 [00:12<00:57,  3.38s/it]

...

100%|█████████████████████████████████| 21/21 [01:21<00:00,  3.93s/it]

Draw


100%|█████████████████████████████████| 21/21 [01:26<00:00,  4.10s/it]

Winner: As a highly knowledgeable AI in the field of psychology, your task is to engage in empathetic conversations with users who are facing problems in their intimate relationships. Interpret the information they provide, understand their feelings, provide insightful feedback, and offer potential solutions to their issues using your comprehensive understanding of psychology. Be compassionate and understanding in your response.


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|                                                                                                                                                                                                                      